In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Underweight/GSE130563'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Skeletal Muscle Fibrosis in Pancreatic Cancer Patients with Respect to Survival"
!Series_summary	"Skeletal muscle wasting is a devastating consequence of cancer that may be responsible for nearly 30% of cancer-related deaths. In addition to muscle atrophy, we have identified significant muscle fiber damage and replacement of muscle with fibrotic tissue in rectus abdominis muscle biopsies from cachectic pancreatic ductal adenocarcinoma (PDAC) patients that associates with poor survival. Transcriptional profiling of muscle harvested from these same patients supported these findings by identifying gene clusters related to wounding, inflammation and cellular response to TGF-B upregulated in cachectic PDAC patients compared with non-cancer controls."
!Series_summary	"In this dataset, we include the expression data obtained from rectus abdominis muscle biopsies fron non-cancer controls patients undergoing abdominal surgery for benign reasons and from PD

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 3  # trait of underweight is inferred from body weight loss prior to surgery
age_row = 4
gender_row = 1

def convert_trait(value):
    try:
        weight_loss = float(value.split(':')[1].strip())
        return 1 if weight_loss > 5 else 0  # Cachexia is defined as body weight loss >5%
    except:
        return None

def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except:
        return None

def convert_gender(value):
    gender = value.split(':')[1].strip()
    if gender == 'F':
        return 0  # female as 0
    elif gender == 'M':
        return 1  # male as 1
    else:
        return None

save_cohort_info('GSE130563', './preprocessed/Underweight/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Underweight', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Underweight/trait_data/GSE130563.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3743555': [0.0, 33.0, 0.0], 'GSM3743556': [0.0, 68.0, 0.0], 'GSM3743557': [0.0, 73.0, 0.0], 'GSM3743558': [0.0, 49.0, 0.0], 'GSM3743559': [0.0, 78.0, 0.0], 'GSM3743560': [0.0, 57.0, 0.0], 'GSM3743561': [0.0, 55.0, 0.0], 'GSM3743562': [0.0, 50.0, 0.0], 'GSM3743563': [0.0, 47.0, 0.0], 'GSM3743564': [0.0, 63.0, 0.0], 'GSM3743565': [0.0, 51.0, 1.0], 'GSM3743566': [0.0, 50.0, 1.0], 'GSM3743567': [1.0, 69.0, 1.0], 'GSM3743568': [0.0, 50.0, 1.0], 'GSM3743569': [1.0, 60.0, 0.0], 'GSM3743570': [1.0, 68.0, 0.0], 'GSM3743571': [1.0, 66.0, 1.0], 'GSM3743572': [1.0, 54.0, 1.0], 'GSM3743573': [1.0, 64.0, 0.0], 'GSM3743574': [1.0, 76.0, 0.0], 'GSM3743575': [1.0, 68.0, 0.0], 'GSM3743576': [1.0, 73.0, 1.0], 'GSM3743577': [1.0, 56.0, 0.0], 'GSM3743578': [1.0, 80.0, 0.0], 'GSM3743579': [1.0, 68.0, 0.0], 'GSM3743580': [1.0, 79.0, 0.0], 'GSM3743581': [1.0, 72.0, 1.0], 'GSM3743582': [1.0, 52.0, 0.0], 'GSM3743583': [1.0, 74.0, 1.0], 'GSM3743584': [1.0, 74.0, 0.0], 'GSM3743585': [1.0, 55.0, 1.0], 'GSM374

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at',
       '100033413_at', '100033422_at', '100033423_at', '100033424_at',
       '100033425_at', '100033426_at', '100033427_at', '100033428_at',
       '100033430_at', '100033431_at', '100033432_at', '100033434_at',
       '100033435_at', '100033436_at', '100033437_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ORF': ['1', '10', '100', '1000', '10000']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify keys from gene annotation data
identifier_key = 'ID'
gene_symbol_key = 'ORF'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to the gene_data dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)
